In [1]:
import gym
import gym_kraby
import math as maths
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from copy import deepcopy as DC
clear_output()

In [2]:
class hexapod:
    def __init__(self):
        self.env= gym.make('gym_kraby:HexapodBulletEnv-v0', render=True)
        obs=self.env.reset()
        self.action_space=self.env.action_space.sample()  # take a random action
        self.sensors={"foot_0":4,"endcap_0":5,"foot_1":10,"endcap_1":11,
            "foot_2":16,"endcap_2": 17,"foot_3": 22,"endcap_3": 23,
            "foot_4": 28,"endcap_4": 29,"foot_5": 34,"endcap_5":35}
    def list_feet(self):
        _link_name_to_index = {self.env.p.getBodyInfo(self.env.robot_id)[0].decode('UTF-8'):-1,}
        for _id in range(self.env.p.getNumJoints(self.env.robot_id)):
            _name = self.env.p.getJointInfo(self.env.robot_id, _id)[12].decode('UTF-8')
            _link_name_to_index[_name] = _id
            if "endcap" in _name or "foot" in _name:
                print(_name,_id)
    def reset(self):
        obs=self.env.reset()
    def step(self,a):
        """
        step through environment with action a
        return reward and the force on the feet that are contacted numbered 0-5 for each leg
        """
        observation, reward, done, _ = self.env.step(a)  # step
        filtered=self.getContact()
        return reward,filtered
    def getAngles(self):
        a=[self.getAngle(i) for i in range(18)]
        return np.array(a)
    def getContact(self):
        VALS=np.array([[5,4],[11,10],[17,16],[23,22],[29,28],[35,34]])
        contact=self.env.p.getContactPoints(bodyA=self.env.robot_id)
        a_=np.array([[5,0],[11,0],[17,0],[23,0],[29,0],[35,0]])
        for c in contact:
            p=c[3]
            f=c[9]
            if p in VALS.flatten():
                id=np.where(VALS == p)
                id=[id[0],id[1]][0]
                if a_[id[0]][0]==p: a_[id[0]][1]=f
                elif a_[id[0]][1]<f: #largest force saves
                    a_[id[0]][0]=p
                    a_[id[0]][1]=f
        return a_
    def getAngle(self,num):
        JOINT=[1,2,4,7,8,10,13,14,16,19,20,22,25,26,28,31,32,34]
        b=self.env.p.getJointState(self.env.robot_id,JOINT[num])
        d=b[0]#convert to degrees
        return (maths.pi/2)+d
    def getImage(self):
        img = self.env.p.getCameraImage(224, 224, renderer=self.env.p.ER_BULLET_HARDWARE_OPENGL)
        return img[2]
    def close(self):
        self.env.close()
    def moveAngles(self,ang):
        for j in range(5): #for accuracy do multiple times
            mov=np.zeros_like(ang)
            angles=self.getAngles()
            for i in range(len(mov)):
                mov[i]=self.moveToAngle(ang[i],angles[i])
            self.step(mov)
            print(np.rad2deg(self.getAngles()))
    def moveToAngle(self,start,to):
        end=start-to
        k=1/np.deg2rad(18)
        return end*k
    

In [3]:
robot=hexapod()


pybullet build time: Jan 20 2023 16:26:58


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=Quadro P620/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 525.105.17
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 525.105.17
Vendor = NVIDIA Corporation
Renderer = Quadro P620/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [250]:
def moveToAngle(start,to):
    end=start-to
    k=1/np.deg2rad(18)
    return end*k
    
def moveAngles(ang):
    for j in range(5): #for accuracy do multiple times
        mov=np.zeros_like(ang)
        angles=robot.getAngles()
        for i in range(len(mov)):
            mov[i]=moveToAngle(ang[i],angles[i])
        robot.step(mov)
        print(np.rad2deg(robot.getAngles()))
    


act=np.ones_like(robot.action_space)*np.deg2rad(80)
moveAngles(act)


[ 79.96637145  80.0134692   80.01217463  79.9909567   79.98308702
  80.0839925   79.99054609  80.02917669  78.95468758  79.96874893
  79.93291959 125.16445046  79.99729097  79.999574    79.98950613
  79.99288102  80.01193171  80.00275042]
[ 79.98517232  79.98177717  79.61811557  80.01519184  79.99715476
  79.11073484  80.01296625  79.99970453  79.9793855   79.9955591
  79.86587388 114.30090651  79.99562944  80.01454492  79.98798585
  79.99039846  80.00619563  79.99882738]
[ 80.01908369  80.0001314   79.49035416  80.05380281  80.01668494
  78.95572599  80.0040594   79.98313437  79.27920662  79.94089363
  80.03183491 110.53133025  79.99796062  79.99015003  80.01249628
  79.97749141  79.99871704  79.80012664]
[ 80.02708232  79.97433709  79.19870841  80.06992494  80.00951033
  78.63683782  80.02287385  80.00775542  80.20084241  79.9883869
  80.09096281 113.88840957  79.99754387  80.01452858  79.99207977
  80.00663958  80.00126884  79.80180928]
[ 80.00304305  79.97851869  79.6890985   80.02

In [240]:
def moveToAngle(start,to):
    end=to-start
    print(np.rad2deg(start),">",np.rad2deg(end))
    k=1/np.deg2rad(18.455)
    return end*k

new=moveToAngle(angles[1],np.deg2rad(140))
print(np.rad2deg(angles[1]),np.rad2deg((new)))
act=np.zeros_like(act)
act[1]=new
robot.step(act)
angles=robot.getAngles()
new=moveToAngle(angles[1],np.deg2rad(140))
print(np.rad2deg(angles[1]),np.rad2deg((new)))

40.16900295896962 > 99.83099704103037
40.16900295896962 309.93740422834196
129.23612972589532 > 10.763870274104658
129.23612972589532 33.41773708656307


In [171]:
robot.reset()
angles=robot.getAngles()
moveBy=10
a=angles[1]
b=angles[2]
act=np.ones_like(robot.action_space) * 1
#act[2]=moveToAngle(b,np.deg2rad(20))
#act=act*-np.pi/2
print(np.rad2deg(a),np.rad2deg(b),np.rad2deg(moveToAngle(a,np.deg2rad(20))))
z=robot.step(act)
angles=robot.getAngles()
c=angles[1]
d=angles[2]
print(np.rad2deg(c),np.rad2deg(d))
print(np.rad2deg(c-a),np.rad2deg(d-b))